In [194]:
import os
import itertools

In [70]:
import numpy as np
import pandas as pd

In [71]:
def check_if_none(*args):
    for arg in args:
        if arg is None:
            raise ValueError('Missing parameter')


def check_if_type(type, *args):
    for arg in args:
        if not isinstance(arg, type):
            raise TypeError('Wrong type of parameter')

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#Read in df1 and df2
df1.astype({"bm": int, "bd": int})
df2.astype({"bm": int, "bd": int})
#pd.merge(df1, df2, how='inner')

In [73]:
import math

__all__ = ['get_jaro_distance']
__author__ = 'Jean-Bernard Ratte - jean.bernard.ratte@unary.ca'

""" Find the Jaro Winkler Distance which indicates the similarity score between two Strings.
    The Jaro measure is the weighted sum of percentage of matched characters from each file and transposed characters.
    Winkler increased this measure for matching initial characters.
    This implementation is based on the Jaro Winkler similarity algorithm from
    http://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
    This Python implementation is based on the Apache StringUtils implementation from
    http://commons.apache.org/proper/commons-lang/apidocs/src-html/org/apache/commons/lang3/StringUtils.html#line.7141
"""


def get_jaro_distance(first, second, winkler=True, winkler_ajustment=True, scaling=0.1):
    """
    :param first: word to calculate distance for
    :param second: word to calculate distance with
    :param winkler: same as winkler_ajustment
    :param winkler_ajustment: add an adjustment factor to the Jaro of the distance
    :param scaling: scaling factor for the Winkler adjustment
    :return: Jaro distance adjusted (or not)
    """
    if not first or not second:
        raise JaroDistanceException("Cannot calculate distance from NoneType ({0}, {1})".format(
            first.__class__.__name__,
            second.__class__.__name__))

    jaro = _score(first, second)
    cl = min(len(_get_prefix(first, second)), 4)

    if all([winkler, winkler_ajustment]):  # 0.1 as scaling factor
        return round((jaro + (scaling * cl * (1.0 - jaro))) * 100.0) / 100.0

    return jaro


def _score(first, second):
    shorter, longer = first.lower(), second.lower()

    if len(first) > len(second):
        longer, shorter = shorter, longer

    m1 = _get_matching_characters(shorter, longer)
    m2 = _get_matching_characters(longer, shorter)

    if len(m1) == 0 or len(m2) == 0:
        return 0.0

    return (float(len(m1)) / len(shorter) +
            float(len(m2)) / len(longer) +
            float(len(m1) - _transpositions(m1, m2)) / len(m1)) / 3.0


def _get_diff_index(first, second):
    if first == second:
        return -1

    if not first or not second:
        return 0

    max_len = min(len(first), len(second))
    for i in range(0, max_len):
        if not first[i] == second[i]:
            return i

    return max_len


def _get_prefix(first, second):
    if not first or not second:
        return ""

    index = _get_diff_index(first, second)
    if index == -1:
        return first

    elif index == 0:
        return ""

    else:
        return first[0:index]


def _get_matching_characters(first, second):
    common = []
    limit = math.floor(min(len(first), len(second)) / 2)

    for i, l in enumerate(first):
        left, right = int(max(0, i - limit)), int(min(i + limit + 1, len(second)))
        if l in second[left:right]:
            common.append(l)
            second = second[0:second.index(l)] + '*' + second[second.index(l) + 1:]

    return ''.join(common)


def _transpositions(first, second):
    return math.floor(len([(f, s) for f, s in zip(first, second) if not f == s]) / 2.0)


class JaroDistanceException(Exception):
    def __init__(self, message):
            super(Exception, self).__init__(message)

In [74]:
def levenshtein_similarity(s1, s2, insert=None, delete=None, substitute=None,
                           insert_default=1, delete_default=1, substitute_default=1):
    """
    Computed as 1 - normalized_levenshtein_distance.
    """
    return 1.0 - normalized_levenshtein_distance(s1, s2, insert, delete, substitute,
                                                 insert_default, delete_default, substitute_default)


def normalized_levenshtein_distance(s1, s2, insert=None, delete=None, substitute=None,
                                    insert_default=1, delete_default=1, substitute_default=1):
    """
    Computed as levenshtein - max-insert-cost(s1,s2)
    """

    insert = insert if isinstance(insert, dict) else {}
    delete = delete if isinstance(delete, dict) else {}
    substitute = substitute if isinstance(substitute, dict) else {}

    def compute_insert_cost(s):
        cost = 0
        for c in s:
            cost += insert[c] if c in insert else insert_default
        return cost

    lev = levenshtein_distance(s1, s2, insert, delete, substitute,
                               insert_default, delete_default, substitute_default)

    max_cost = max(compute_insert_cost(s1), compute_insert_cost(s2))

    if max_cost < lev:
        raise ValueError('Illegal value of operation cost')

    if max_cost == 0:
        return 0

    return float(lev) / max_cost

def levenshtein_distance(s1, s2, insert=None, delete=None, substitute=None,
                         insert_default=1, delete_default=1, substitute_default=1):
    """
    The Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.
    Args:
        s1 (str): Sequence 1.
        s2 (str): Sequence 2.
        insert (dict(str, int), optional): Insert cost of characters. Defaults to None.
        delete (dict(str, int), optional): Delete cost of characters. Defaults to None.
        substitute (dict(str, dict(str, int)), optional): Substitute cost of characters. Defaults to None.
        insert_default (int, optional): Default value of insert cost. Defaults to 1.
        delete_default (int, optional): Default value of delete cost. Defaults to 1.
        substitute_default (int, optional): Default value of substitute cost. Defaults to 1.
    Returns:
        int: Levenshtein Distance.
    Examples:
        >>> rltk.levenshtein_distance('ab', 'abc')
        1
        >>> rltk.levenshtein_distance('a', 'abc', insert = {'c':50},
        ... insert_default=100, delete_default=100, substitute_default=100)
        150
    """

    check_if_none(s1, s2)
    check_if_type(str, s1, s2)

    insert = insert if isinstance(insert, dict) else {}
    delete = delete if isinstance(delete, dict) else {}
    substitute = substitute if isinstance(substitute, dict) else {}

    # s1 = utils.unicode_normalize(s1)
    # s2 = utils.unicode_normalize(s2)

    n1, n2 = len(s1), len(s2)
    if n1 == 0 and n2 == 0:
        return 0

    # if n1 == 0 or n2 == 0:
    #     return max(n1, n2)

    dp = [[0] * (n2 + 1) for _ in range(n1 + 1)]
    for i in range(n1 + 1):
        for j in range(n2 + 1):
            if i == 0 and j == 0:  # [0,0]
                continue
            elif i == 0:  # most top row
                c = s2[j - 1]
                dp[i][j] = insert[c] if c in insert else insert_default
                dp[i][j] += dp[i][j - 1]
            elif j == 0:  # most left column
                c = s1[i - 1]
                dp[i][j] = delete[c] if c in delete else delete_default
                dp[i][j] += dp[i - 1][j]
            else:
                c1, c2 = s1[i - 1], s2[j - 1]
                insert_cost = insert[c2] if c2 in insert else insert_default
                delete_cost = delete[c1] if c1 in delete else delete_default
                substitute_cost = substitute[c1][c2] \
                    if c1 in substitute and c2 in substitute[c1] else substitute_default

                if c1 == c2:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = min(dp[i][j - 1] + insert_cost,
                                   dp[i - 1][j] + delete_cost,
                                   dp[i - 1][j - 1] + substitute_cost)
    return dp[n1][n2]

In [206]:
def generate_pairs(dfA,dfB, key_col, attr_fn_mapping = {}, method = 'jaro', threshold=0.7):

  nB,k = dfB.shape
  
  sim_mat = []
  for idx in range(nB):
    sim_mat.append(list(dfA.apply(_compare ,axis =1, rowB = dfB.iloc[idx],key_col = 'id', attr_fn_mapping = attr_fn_mapping )))

  attrs =  ['ID1','ID2'] + [ attr for attr in list(df2.columns) if attr != key_col]
  return pd.DataFrame(list(itertools.chain(*sim_mat)), columns=attrs)


In [133]:
def _compare(rowA, rowB, key_col , attr_fn_mapping= {}, method='jaro', threshold = 0.7):
  agg_row = [getattr(rowA, key_col),getattr(rowB, key_col)]
  for attr,val in rowA.iteritems():
    if attr != key_col :
      if isinstance(val,str):
        if method == 'jaro':
          _sim = get_jaro_distance(val,getattr(rowB, attr))
        elif method == 'levenshtein':
          _sim = levenshtein_similarity(val,getattr(rowB, attr))
        else:
          _sim = 0.0
      elif val == getattr(rowB, attr):
        _sim = 1.0
      else:
        _sim = 0.0
      agg_row.append(_sim)

  return agg_row
